In [1]:
# ============================================================
# CELL 1: IMPORTS & SETUP
# ============================================================

# --- PATH SETUP (MUST BE FIRST) ---
import sys
from pathlib import Path

# Get current directory (should be notebooks/)
current_dir = Path.cwd()
print(f"📁 Current directory: {current_dir}")
print(f"📁 Current directory name: {current_dir.name}")

# Go up one level to project root
project_root = current_dir.parent
print(f"📁 Project root: {project_root}")

# Verify database folder exists
database_path = project_root / 'database'
print(f"📁 Database path: {database_path}")
print(f"✅ Database folder exists: {database_path.exists()}")

# Add project root to Python path
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
    print(f"✅ Added to Python path: {project_root}")
else:
    print(f"✅ Already in Python path")

print(f"\n📋 Python path (first 3):")
for i, p in enumerate(sys.path[:3], 1):
    print(f"   {i}. {p}")


# --- STANDARD LIBRARY IMPORTS ---
import os
import json
import time
import sqlite3
import glob
from typing import TypedDict, Optional, List, Dict, Any
from datetime import datetime
from enum import Enum

print("✅ Standard library imports loaded")


# --- ENVIRONMENT & CONFIGURATION ---
from dotenv import load_dotenv

# Load environment variables from project root
env_path = project_root / '.env'
if env_path.exists():
    load_dotenv(env_path)
    print(f"✅ Loaded .env from: {env_path}")
else:
    print(f"⚠️  .env not found at: {env_path}")


# --- THIRD-PARTY ML/AI ---
import assemblyai as aai
from langchain_anthropic import ChatAnthropic
from langgraph.graph import StateGraph
from pydantic import BaseModel, Field

print("✅ Third-party imports loaded")


# --- DATABASE IMPORTS ---
from database.db_operations import db
from database.models import Conversation, ConversationCreate

print("✅ Database imports loaded")
print("\n🎉 All imports loaded successfully!")

📁 Current directory: /home/manuel/Documents/tech/ai_content_ops/notebooks
📁 Current directory name: notebooks
📁 Project root: /home/manuel/Documents/tech/ai_content_ops
📁 Database path: /home/manuel/Documents/tech/ai_content_ops/database
✅ Database folder exists: True
✅ Added to Python path: /home/manuel/Documents/tech/ai_content_ops

📋 Python path (first 3):
   1. /home/manuel/Documents/tech/ai_content_ops
   2. /usr/lib/python313.zip
   3. /usr/lib/python3.13
✅ Standard library imports loaded
✅ Loaded .env from: /home/manuel/Documents/tech/ai_content_ops/.env
✅ Third-party imports loaded
✅ Database imports loaded

🎉 All imports loaded successfully!


In [2]:

# Cell 2 

# Test API key
assemblyai_key = os.getenv('ASSEMBLYAI_API_KEY')
print(f"AssemblyAI API Key loaded: {'✅' if assemblyai_key else '❌'}")
print(f"Key starts with: {assemblyai_key[:10] if assemblyai_key else 'None'}...")


AssemblyAI API Key loaded: ✅
Key starts with: 972365f41d...


In [3]:
# ============================================================
# CELL 3: LANGSMITH TRACING SETUP (OPTIONAL)
# ============================================================

import os
from dotenv import load_dotenv

# Load environment variables if not already loaded
load_dotenv()

# Get LangSmith API key from environment
langsmith_api_key = os.getenv("LANGSMITH_API_KEY")

if langsmith_api_key:
    # Enable LangSmith tracing
    os.environ["LANGSMITH_API_KEY"] = langsmith_api_key
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    os.environ["LANGCHAIN_PROJECT"] = "ai_content_ops"
    print("✅ LangSmith tracing enabled")
    print(f"   Project: ai_content_ops")
else:
    print("⚠️  LANGSMITH_API_KEY not found in .env")
    print("   LangSmith tracing disabled")
    print("   💡 Add LANGSMITH_API_KEY to your .env file to enable tracing")

✅ LangSmith tracing enabled
   Project: ai_content_ops


In [4]:
# Cell 3: Database Connection Test

conn = sqlite3.connect("data/app.db")
cursor = conn.cursor()

# Get all table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print("📊 Tables in app.db:")
for table in tables:
    table_name = table[0]
    
    # Get column info for each table
    cursor.execute(f"PRAGMA table_info({table_name})")
    columns = cursor.fetchall()
    
    print(f"\n🔧 {table_name}:")
    for col in columns:
        print(f"   - {col[1]} ({col[2]})")  # column_name (type)

conn.close()

📊 Tables in app.db:

🔧 conversations:
   - id (INTEGER)
   - title (TEXT)
   - raw_text (TEXT)
   - source (TEXT)
   - word_count (INTEGER)
   - created_at (DATETIME)
   - status (TEXT)

🔧 sqlite_sequence:
   - name ()
   - seq ()

🔧 blog_post_ideas:
   - id (INTEGER)
   - conversation_id (INTEGER)
   - title (TEXT)
   - description (TEXT)
   - usefulness_potential (INTEGER)
   - fitwith_seo_strategy (INTEGER)
   - fitwith_content_strategy (INTEGER)
   - inspiration_potential (INTEGER)
   - collaboration_potential (INTEGER)
   - innovation (INTEGER)
   - difficulty (INTEGER)
   - total_score (INTEGER)
   - sent_to_prod (BOOLEAN)
   - raw_llm_response (TEXT)
   - created_at (DATETIME)

🔧 processing_status:
   - id (INTEGER)
   - conversation_id (INTEGER)
   - stage (TEXT)
   - status (TEXT)
   - error_message (TEXT)
   - started_at (DATETIME)
   - completed_at (DATETIME)


In [5]:
## cell 4. Pydantic Model for Structured Output



class SpeakerRole(str, Enum):
    """Possible speaker roles in the conversation"""
    CLIENT = "client"
    INTERVIEWER = "interviewer"

class Speaker(BaseModel):
    """Information about a person speaking in the conversation"""
    name: Optional[str] = Field(default=None, description="Name of the speaker if mentioned")
    role: Optional[SpeakerRole] = Field(default=None, description="Role of the speaker in the conversation")
    company: Optional[str] = Field(default=None, description="Company they work for if mentioned")

class Challenge(BaseModel):
    """A challenge or problem mentioned in the conversation"""
    description: Optional[str] = Field(default=None, description="Description of the challenge")
    impact: Optional[str] = Field(default=None, description="How this challenge affects them")
    urgency: Optional[str] = Field(default=None, description="Low, Medium, or High urgency")

class CurrentSolution(BaseModel):
    """How they currently solve their problems"""
    solution: Optional[str] = Field(default=None, description="What they're currently doing")
    satisfaction_level: Optional[str] = Field(default=None, description="How satisfied they are: Very Satisfied, Satisfied, Neutral, Unsatisfied, Very Unsatisfied")
    limitations: Optional[List[str]] = Field(default=[], description="Limitations of current solution")

class Need(BaseModel):
    """A need identified using psychology frameworks like NVC"""
    need_category: Optional[str] = Field(default=None, description="Category of need (e.g., autonomy, efficiency, security, connection)")
    description: Optional[str] = Field(default=None, description="Specific need description")
    intensity: Optional[str] = Field(default=None, description="Low, Medium, or High intensity")

class ExtractedInsights(BaseModel):
    """Complete structured output from conversation analysis"""
    
    # Speakers
    speakers: Optional[List[Speaker]] = Field(default=[], description="People identified in the conversation")
    
    # What they care about
    core_values: Optional[List[str]] = Field(default=[], description="What this person/company cares about most")
    priorities: Optional[List[str]] = Field(default=[], description="Their current priorities and focus areas")
    
    # Challenges
    primary_challenges: Optional[List[Challenge]] = Field(default=[], description="Main problems they're facing")
    secondary_challenges: Optional[List[Challenge]] = Field(default=[], description="Secondary or related problems")
    
    # Current solutions
    current_solutions: Optional[List[CurrentSolution]] = Field(default=[], description="How they solve problems today")
    
    # Needs analysis
    psychological_needs: Optional[List[Need]] = Field(default=[], description="Underlying needs using NVC or similar frameworks")
 


In [6]:
# Cell 5: Raw Blog Idea Model (Simple)
class RawBlogIdea(BaseModel):
    """Raw blog idea from creative agent"""
    title: str
    description: str
    target_audience: str
    content_angle: str
    business_value: str

print("✅ Simple RawBlogIdea model ready")

✅ Simple RawBlogIdea model ready


In [7]:
def validate_raw_blog_ideas(raw_ideas: List[Dict]) -> List[RawBlogIdea]:
    """Validate and convert raw JSON to Pydantic models"""
    validated_ideas = []
    
    for idea in raw_ideas:
        try:
            validated_idea = RawBlogIdea(**idea)
            validated_ideas.append(validated_idea)
        except Exception as e:
            print(f"⚠️ Invalid blog idea skipped: {e}")
    
    print(f"✅ Validated {len(validated_ideas)} out of {len(raw_ideas)} raw ideas")
    return validated_ideas

print("✅ RawBlogIdea model and validation ready")

✅ RawBlogIdea model and validation ready


In [8]:
# Cell 6: Update AudioPipelineState for 5-Node Pipeline
class AudioPipelineState(TypedDict):
    # File info
    file_path: str
    filename: str
    
    # Processing results
    transcript_text: Optional[str]
    conversation_id: Optional[int]
    extracted_insights: Optional[ExtractedInsights]
    raw_blog_ideas: Optional[List[Dict]]        # From creative agent (Node 4)
    scored_blog_ideas: Optional[List[Dict]]     # From analyst agent (Node 5) ← NEW
    
    
    # Status & error handling
    status: str
    error: Optional[str]

print("✅ Updated AudioPipelineState for 5-node pipeline")

✅ Updated AudioPipelineState for 5-node pipeline


In [9]:
# Cell 7: Updated Company Strategy Context Loader (3 Documents)
def load_company_strategy_context():
    """Load company strategy, SEO strategy, and content strategy for context"""
    
    strategy_context = {}
    
    try:
        # Load company strategy
        company_strategy_path = "../data/processed/company_strategy.mkd"
        if os.path.exists(company_strategy_path):
            with open(company_strategy_path, "r", encoding="utf-8") as f:
                strategy_context["company_strategy"] = f.read()
            print(f"✅ Loaded company strategy ({len(strategy_context['company_strategy'])} chars)")
        else:
            strategy_context["company_strategy"] = "Company strategy document not available."
            print("⚠️ Company strategy document not found")
        
        # Load SEO strategy
        seo_strategy_path = "../data/processed/seo_strategy.mkd"
        if os.path.exists(seo_strategy_path):
            with open(seo_strategy_path, "r", encoding="utf-8") as f:
                strategy_context["seo_strategy"] = f.read()
            print(f"✅ Loaded SEO strategy ({len(strategy_context['seo_strategy'])} chars)")
        else:
            strategy_context["seo_strategy"] = "SEO strategy document not available."
            print("⚠️ SEO strategy document not found")
        
        # Load content strategy (NEW)
        content_strategy_path = "../data/processed/content_strategy.mkd"
        if os.path.exists(content_strategy_path):
            with open(content_strategy_path, "r", encoding="utf-8") as f:
                strategy_context["content_strategy"] = f.read()
            print(f"✅ Loaded content strategy ({len(strategy_context['content_strategy'])} chars)")
        else:
            strategy_context["content_strategy"] = "Content strategy document not available."
            print("⚠️ Content strategy document not found")
            
    except Exception as e:
        print(f"❌ Error loading strategy documents: {e}")
        strategy_context = {
            "company_strategy": "Strategy document not available",
            "seo_strategy": "SEO strategy document not available", 
            "content_strategy": "Content strategy document not available"
        }
    
    return strategy_context

# Test loading all three documents
strategy_context = load_company_strategy_context()
print(f"📊 Strategy context keys: {list(strategy_context.keys())}")
print(f"📊 Total context size: {sum(len(v) for v in strategy_context.values() if isinstance(v, str))} chars")

✅ Loaded company strategy (6555 chars)
✅ Loaded SEO strategy (1120 chars)
✅ Loaded content strategy (4469 chars)
📊 Strategy context keys: ['company_strategy', 'seo_strategy', 'content_strategy']
📊 Total context size: 12144 chars


In [10]:
# Batch Processing Function (Updated with Full Insights Display)
def process_audio_batch(audio_files: List[Path], pipeline) -> dict:
    """Process all audio files in batch with detailed insights display"""
    
    if not audio_files:
        print("❌ No files to process")
        return {"processed": [], "failed": [], "total": 0}
    
    print(f"\n🚀 STARTING BATCH PROCESSING - {len(audio_files)} files")
    print("=" * 60)
    
    processed_files = []
    failed_files = []
    results = []
    
    for i, file_path in enumerate(audio_files, 1):
        print(f"\n📂 Processing {i}/{len(audio_files)}: {file_path.name}")
        print("-" * 40)
        
        # Create initial state
        initial_state = {
            "file_path": str(file_path),
            "filename": file_path.name,
            "transcript_text": None,
            "conversation_id": None,
            "extracted_insights": None,  
            "error": None,
            "status": "processing"
        }
        
        try:
            # Run through pipeline
            result = pipeline.invoke(initial_state)
            
            if result["status"] in ["completed", "insights_extracted"]:
                print(f"✅ SUCCESS: {file_path.name}")
                print(f"   Conversation ID: {result['conversation_id']}")
                print(f"   Transcript preview: {result['transcript_text'][:100]}...")
                
                # FULL INSIGHTS DISPLAY
                if result.get('extracted_insights'):
                    insights = result['extracted_insights']
                    print(f"\n🧠 === EXTRACTED INSIGHTS FOR: {file_path.name} ===")
                    print("=" * 50)
                    
                    # Speakers
                    if insights.speakers:
                        print("👥 SPEAKERS:")
                        for speaker in insights.speakers:
                            print(f"   • Name: {speaker.name or 'Unknown'}")
                            print(f"     Role: {speaker.role or 'Unknown'}")  
                            print(f"     Company: {speaker.company or 'Unknown'}")
                    
                    # Core Values
                    if insights.core_values:
                        print("💎 CORE VALUES:")
                        for value in insights.core_values:
                            print(f"   • {value}")
                    
                    # Priorities
                    if insights.priorities:
                        print("🎯 PRIORITIES:")
                        for priority in insights.priorities:
                            print(f"   • {priority}")
                    
                    # Primary Challenges
                    if insights.primary_challenges:
                        print("🔥 PRIMARY CHALLENGES:")
                        for challenge in insights.primary_challenges:
                            print(f"   • Challenge: {challenge.description}")
                            print(f"     Impact: {challenge.impact}")
                            print(f"     Urgency: {challenge.urgency}")
                    
                    # Secondary Challenges
                    if insights.secondary_challenges:
                        print("⚠️  SECONDARY CHALLENGES:")
                        for challenge in insights.secondary_challenges:
                            print(f"   • Challenge: {challenge.description}")
                            print(f"     Impact: {challenge.impact}")
                            print(f"     Urgency: {challenge.urgency}")
                    
                    # Current Solutions
                    if insights.current_solutions:
                        print("🔧 CURRENT SOLUTIONS:")
                        for solution in insights.current_solutions:
                            print(f"   • Solution: {solution.solution}")
                            print(f"     Satisfaction: {solution.satisfaction_level}")
                            if solution.limitations:
                                print(f"     Limitations: {', '.join(solution.limitations)}")
                    
                    # Psychological Needs
                    if insights.psychological_needs:
                        print("🧘 PSYCHOLOGICAL NEEDS:")
                        for need in insights.psychological_needs:
                            print(f"   • {need.description}")
                            print(f"     Category: {need.need_category}")
                            print(f"     Intensity: {need.intensity}")
                    
                    print("🧠 === END INSIGHTS ===")
                    print("-" * 50)
                
                processed_files.append(file_path)
            else:
                print(f"❌ FAILED: {file_path.name}")
                print(f"   Status: {result.get('status', 'Unknown')}")
                print(f"   Error: {result.get('error', 'Unknown error')}")
                failed_files.append(file_path)
            
            results.append(result)
            
        except Exception as e:
            print(f"❌ PIPELINE ERROR: {file_path.name}")
            print(f"   Exception: {str(e)}")
            failed_files.append(file_path)
            
            results.append({
                **initial_state,
                "error": str(e),
                "status": "pipeline_error"
            })
    
    # Final Summary
    print(f"\n📊 BATCH PROCESSING COMPLETE!")
    print("=" * 60)
    print(f"✅ Successfully processed: {len(processed_files)}")
    print(f"❌ Failed: {len(failed_files)}")
    print(f"📁 Total files: {len(audio_files)}")
    
    if failed_files:
        print(f"\n❌ Failed files:")
        for failed_file in failed_files:
            print(f"   - {failed_file.name}")
    
    return {
        "processed": processed_files,
        "failed": failed_files,
        "total": len(audio_files),
        "results": results
    }

print("✅ Batch processing function ready with full insights display")

✅ Batch processing function ready with full insights display


In [11]:
# Cell 5: Batch File Discovery and Management


def find_audio_files_in_temp(temp_folder: Path = None) -> List[Path]:
    """Find all audio files in temp folder"""
    
    # Use default temp folder if not specified
    if temp_folder is None:
        temp_folder = project_root / 'data' / 'temp'
    
    # Ensure folder exists
    temp_folder.mkdir(parents=True, exist_ok=True)
    
    # Check if folder exists
    if not temp_folder.exists():
        print(f"❌ Temp folder not found: {temp_folder}")
        return []
    
    # Find audio files
    audio_extensions = ['*.wav', '*.mp3', '*.m4a']
    audio_files = []
    
    for ext in audio_extensions:
        files = list(temp_folder.glob(ext))
        audio_files.extend(files)
    
    return sorted(audio_files)


def display_batch_info(audio_files: List[Path]) -> bool:
    """Display information about the batch of files"""
    
    if not audio_files:
        print("❌ No audio files found in temp folder!")
        print("💡 TIP: Add .wav files to data/temp/ folder")
        return False
    
    total_size_mb = sum(f.stat().st_size for f in audio_files) / (1024 * 1024)
    
    print(f"📊 BATCH INFO:")
    print(f"   Files to process: {len(audio_files)}")
    print(f"   Total size: {total_size_mb:.1f} MB")
    print(f"\n📁 Files found:")
    
    for i, file_path in enumerate(audio_files, 1):
        size_mb = file_path.stat().st_size / (1024 * 1024)
        print(f"   {i}. {file_path.name} ({size_mb:.1f} MB)")
    
    return True


print("✅ File management functions defined")

✅ File management functions defined


In [12]:
# Cell: Extract Insights Function - ROLE FIXED VERSION
def extract_insights_from_transcript(transcript: str) -> ExtractedInsights:
    """Extract structured insights using Anthropic Claude - ROLE FIXED VERSION"""
    
    prompt = f"""
    Analyze this conversation transcript and extract structured insights:
    
    Transcript: {transcript}
    
    IMPORTANT: For speaker roles, use ONLY these exact values:
    - "client" for the person being interviewed/consulted (CTO, CEO, Manager, business owner, etc.)
    - "interviewer" for the person asking questions or conducting the interview
    
    Extract the following information in JSON format:
    - speakers: List of people mentioned with name, role (client/interviewer only), company
    - core_values: What they care about most  
    - priorities: Current focus areas
    - primary_challenges: Main problems they face with description, impact, urgency
    - secondary_challenges: Secondary problems
    - current_solutions: How they solve problems now with satisfaction level
    - psychological_needs: Underlying needs with category, description, intensity
    
    Return ONLY valid JSON in this exact structure - no markdown, no code blocks:
    {{
        "speakers": [
            {{"name": "Manuel", "role": "client", "company": "Drone flytech"}}
        ],
        "core_values": ["efficiency", "transparency"],
        "priorities": ["improving processes"],
        "primary_challenges": [
            {{
                "description": "Tracking payment issues",
                "impact": "Creates confusion in processes", 
                "urgency": "High"
            }}
        ],
        "secondary_challenges": [
            {{
                "description": "Secondary challenge",
                "impact": "Secondary impact",
                "urgency": "Medium"
            }}
        ],
        "current_solutions": [
            {{
                "solution": "Current approach",
                "satisfaction_level": "Neutral",
                "limitations": ["limitation1", "limitation2"]
            }}
        ],
        "psychological_needs": [
            {{
                "need_category": "security",
                "description": "Need for confidence",
                "intensity": "High"
            }}
        ]
    }}
    
    Remember: 
    - Use "client" for Manuel (even though he's CTO)
    - Use "interviewer" for the person asking questions
    - Use exact urgency values: "Low", "Medium", "High"
    - Use exact satisfaction levels: "Very Satisfied", "Satisfied", "Neutral", "Unsatisfied", "Very Unsatisfied"
    - Use exact intensity values: "Low", "Medium", "High"
    """
    
    try:
        # Use the Claude LLM you already set up
        response = llm.invoke(prompt)
        
        print(f"📝 Raw response length: {len(response.content)} chars")
        print(f"📝 Response starts with: {response.content[:50]}...")
        
        # Clean markdown code blocks
        content = response.content.strip()
        if content.startswith('```json'):
            print("🔧 Removing JSON markdown blocks...")
            content = content.replace('```json', '').replace('```', '').strip()
            print(f"🔧 Cleaned content starts with: {content[:50]}...")
        
        # Parse the cleaned JSON response
        insights_data = json.loads(content)
        
        # Convert to Pydantic model
        result = ExtractedInsights(**insights_data)
        print(f"✅ Successfully extracted insights with correct speaker roles!")
        return result
        
    except json.JSONDecodeError as e:
        print(f"❌ JSON parsing error: {e}")
        print(f"📝 Raw response: {response.content[:500]}...")
        raise
    except Exception as e:
        print(f"❌ Error in LLM call: {e}")
        raise

print("✅ Updated extract_insights_from_transcript with speaker role fix")

✅ Updated extract_insights_from_transcript with speaker role fix


In [13]:
# Cell: Fixed Creative Agent Function
def generate_blog_ideas_from_insights(insights: ExtractedInsights, strategy_context: dict) -> List[Dict]:
    """
    Fixed creative agent that handles Claude's markdown JSON response
    """
    
    creative_prompt = f"""
    You are a creative content strategist for Big Kids Automation, a company that helps businesses implement AI and automation solutions.
    
    COMPANY CONTEXT:
    {strategy_context.get('company_strategy', 'Strategy not available')[:1000]}...
    
    SEO STRATEGY:
    {strategy_context.get('seo_strategy', 'SEO strategy not available')[:500]}...
    
    CONVERSATION INSIGHTS TO WORK FROM:
    
    Speakers: {[f"{s.name} ({s.role}) from {s.company}" for s in insights.speakers] if insights.speakers else "Unknown speakers"}
    
    Core Values: {", ".join(insights.core_values) if insights.core_values else "None identified"}
    
    Priorities: {", ".join(insights.priorities) if insights.priorities else "None identified"}
    
    Primary Challenges:
    {chr(10).join([f"- {c.description} (Impact: {c.impact}, Urgency: {c.urgency})" for c in insights.primary_challenges]) if insights.primary_challenges else "None identified"}
    
    Current Solutions:
    {chr(10).join([f"- {s.solution} (Satisfaction: {s.satisfaction_level})" for s in insights.current_solutions]) if insights.current_solutions else "None identified"}
    
    Psychological Needs:
    {chr(10).join([f"- {n.description} ({n.need_category}, {n.intensity} intensity)" for n in insights.psychological_needs]) if insights.psychological_needs else "None identified"}
    
    TASK:
    Generate 4-5 creative blog post ideas that:
    1. Address the challenges and needs identified in this conversation
    2. Align with Big Kids Automation's mission to help businesses with AI/automation
    3. Provide value to potential clients facing similar challenges
    4. Support our SEO and content marketing strategy
    5. Are actionable and practical, not just theoretical
    
    For each blog post idea, provide:
    - title: Clear, engaging title that includes relevant keywords
    - description: 2-3 sentence description of what the post will cover
    - target_audience: Who this post is primarily for
    - content_angle: The unique angle or approach this post takes
    - business_value: How this post helps our business goals
    
    IMPORTANT: Return ONLY the JSON array, no markdown formatting, no code blocks, no explanatory text.
    
    Format:
    [
        {{
            "title": "How AI Proposal Systems Balance Speed with Brand Differentiation",
            "description": "A practical guide showing how modern AI-powered proposal systems solve the common problem of maintaining company uniqueness while leveraging automation. Includes real case studies and implementation steps.",
            "target_audience": "Business development directors and proposal managers at consulting firms",
            "content_angle": "Problem-solution with real case studies",
            "business_value": "Attracts prospects struggling with proposal automation while maintaining differentiation"
        }}
    ]
    """
    
    try:
        # Generate ideas using Claude
        response = llm.invoke(creative_prompt)
        raw_content = response.content.strip()
        
        print(f"📝 Raw response length: {len(raw_content)} chars")
        print(f"📝 Response starts with: {raw_content[:50]}...")
        
        # Handle markdown code blocks
        if raw_content.startswith('```'):
            print("🔧 Removing markdown code blocks...")
            # Remove ```json and ``` wrappers
            lines = raw_content.split('\n')
            # Remove first line if it's ```json or ```
            if lines[0].startswith('```'):
                lines = lines[1:]
            # Remove last line if it's ```
            if lines and lines[-1].strip() == '```':
                lines = lines[:-1]
            raw_content = '\n'.join(lines).strip()
            print(f"🔧 Cleaned content starts with: {raw_content[:50]}...")
        
        # Parse JSON response
        blog_ideas = json.loads(raw_content)
        
        print(f"✅ Creative agent successfully parsed {len(blog_ideas)} blog ideas")
        return blog_ideas
        
    except json.JSONDecodeError as e:
        print(f"❌ JSON parsing error in creative agent: {e}")
        print(f"📝 Cleaned content: {raw_content[:500]}...")
        return []
    except Exception as e:
        print(f"❌ Error in creative agent: {e}")
        return []

print("✅ Fixed creative agent function ready")

✅ Fixed creative agent function ready


In [14]:
# Cell 17: Updated Scoring Engine with Content Strategy Context
def score_blog_idea_with_llm(idea: dict, strategy_context: dict, conversation_context: str = "") -> dict:
    """Score a single blog idea using LLM with all three strategy contexts"""
    
    scoring_prompt = f"""
    You are an expert content strategist for Big Kids Automation. Score this blog post idea on a 1-10 scale using our strategic context.
    
    COMPANY STRATEGY:
    {strategy_context.get('company_strategy_summary', 'Not available')}
    
    SEO STRATEGY:
    {strategy_context.get('seo_strategy_summary', 'Not available')}
    
    CONTENT STRATEGY:
    {strategy_context.get('content_strategy_summary', 'Not available')}
    
    BLOG IDEA TO SCORE:
    Title: {idea.get('title', 'No title')}
    Description: {idea.get('description', 'No description')}
    Target Audience: {idea.get('target_audience', 'Unknown')}
    Business Value: {idea.get('business_value', 'Unknown')}
    Content Angle: {idea.get('content_angle', 'Unknown')}
    
    CONVERSATION CONTEXT:
    {conversation_context[:300] if conversation_context else 'No context available'}...
    
    SCORING INSTRUCTIONS:
    Rate each criterion from 1-10 (10 = excellent, 1 = poor):
    
    1. usefulness_potential: How useful will this be to readers with real problems?
    2. fitwith_seo_strategy: How well does this align with our SEO keywords and strategy?
    3. fitwith_content_strategy: How well does this fit our content strategy, voice, and approach?
    4. inspiration_potential: How likely to inspire readers to take meaningful action?
    5. collaboration_potential: How likely to generate leads/prospects who contact us?
    6. innovation: How unique is this topic compared to existing content?
    7. difficulty: How complex/time-consuming will this be to write? (1=very hard, 10=easy)
    
    Return ONLY valid JSON with your scores and brief reasoning:
    {{
        "usefulness_potential": 8,
        "fitwith_seo_strategy": 7,
        "fitwith_content_strategy": 9,
        "inspiration_potential": 6,
        "collaboration_potential": 8,
        "innovation": 7,
        "difficulty": 4,
        "reasoning": "This idea scores well because it aligns with our content strategy focus on..."
    }}
    """
    
    # ... rest of the function stays the same
    try:
        response = llm.invoke(scoring_prompt)
        
        content = response.content.strip()
        if content.startswith('```json'):
            content = content.replace('```json', '').replace('```', '').strip()
        
        scores = json.loads(content)
        
        # Validate scores are in range
        for criterion in ['usefulness_potential', 'fitwith_seo_strategy', 'fitwith_content_strategy', 
                         'inspiration_potential', 'collaboration_potential', 'innovation', 'difficulty']:
            if criterion in scores:
                scores[criterion] = max(1, min(10, scores[criterion]))
        
        # Calculate total score
        total_score = sum([
            scores.get('usefulness_potential', 5),
            scores.get('fitwith_seo_strategy', 5),
            scores.get('fitwith_content_strategy', 5),
            scores.get('inspiration_potential', 5),
            scores.get('collaboration_potential', 5),
            scores.get('innovation', 5),
            scores.get('difficulty', 5)
        ])
        
        scores['total_score'] = total_score
        return scores
        
    except Exception as e:
        print(f"❌ Error scoring idea: {e}")
        return {
            "usefulness_potential": 5, "fitwith_seo_strategy": 5, "fitwith_content_strategy": 5,
            "inspiration_potential": 5, "collaboration_potential": 5, "innovation": 5,
            "difficulty": 5, "total_score": 35, "reasoning": f"Default scores due to error: {str(e)}"
        }

print("✅ Updated LLM scoring engine with content strategy context")

✅ Updated LLM scoring engine with content strategy context


In [15]:
# Cell 6: Define LangGraph Nodes
def transcription_node(state: AudioPipelineState) -> AudioPipelineState:
    """Node 1: Transcribe audio file with AssemblyAI"""
    try:
        print(f"🎙️ Transcribing: {state['filename']}")
        
        # Configure transcriber
        transcriber = aai.Transcriber()
        
        # Transcribe the file
        transcript = transcriber.transcribe(state['file_path'])
        
        if transcript.status == aai.TranscriptStatus.error:
            return {
                **state,
                "error": f"AssemblyAI error: {transcript.error}",
                "status": "transcription_failed"
            }
        
        return {
            **state,
            "transcript_text": transcript.text,
            "status": "transcribed"
        }
        
    except Exception as e:
        return {
            **state,
            "error": f"Transcription error: {str(e)}",
            "status": "transcription_failed"
        }

def database_saver_node(state: AudioPipelineState) -> AudioPipelineState:
    """Node 2: Save transcript to database"""
    try:
        print(f"💾 Saving to database: {state['filename']}")
        
        # Create conversation object
        conversation = ConversationCreate(
            title=f"Audio: {state['filename']}",
            raw_text=state['transcript_text'],
            source="transcribed"
        )
        
        # Save to database
        conversation_id = db.create_conversation(conversation)
        
        return {
            **state,
            "conversation_id": conversation_id,
            "status": "completed"
        }
        
    except Exception as e:
        return {
            **state,
            "error": f"Database error: {str(e)}",
            "status": "database_failed"
        }

print("✅ LangGraph nodes defined")

✅ LangGraph nodes defined


In [16]:
def pain_extractor_node(state: AudioPipelineState) -> AudioPipelineState:
    """
    LangGraph node: Extract structured insights from conversation transcript
    """
    print("🧠 Starting pain extraction...")
    
    try:
        # Extract insights using OpenAI structured output
        insights = extract_insights_from_transcript(state['transcript_text'])
        
        if insights:
            print(f"✅ Extracted insights: {len(insights.primary_challenges)} primary challenges, {len(insights.speakers)} speakers")
            
            return {
                **state,
                "extracted_insights": insights,
                "status": "insights_extracted"
            }
        else:
            return {
                **state,
                "error": "Failed to extract insights from transcript",
                "status": "error"
            }
            
    except Exception as e:
        print(f"❌ Pain extraction failed: {e}")
        return {
            **state,
            "error": f"Pain extraction error: {str(e)}",
            "status": "error"
        }

In [17]:
# Cell: Creative Agent Node - FORCE RELOAD
def creative_agent_node(state: AudioPipelineState) -> AudioPipelineState:
    """Creative agent that generates raw blog ideas"""
    
    try:
        print("🎨 Starting creative blog idea generation...")
        
        insights = state.get('extracted_insights')
        if not insights:
            return {**state, "error": "No insights available", "status": "error"}
        
        print(f"📊 Working with insights: {len(insights.primary_challenges)} challenges")
        
        # Load strategy context
        strategy_context = load_company_strategy_context()
        
        # Generate ideas (returns JSON list)
        raw_ideas_json = generate_blog_ideas_from_insights(insights, strategy_context)
        
        if not raw_ideas_json:
            return {**state, "error": "No ideas generated", "status": "error"}
        
        # Convert to Pydantic for validation
        validated_ideas = []
        for idea_json in raw_ideas_json:
            try:
                idea = RawBlogIdea(**idea_json)
                validated_ideas.append(idea)
            except Exception as e:
                print(f"⚠️ Skipping invalid idea: {e}")
        
        if validated_ideas:
            print(f"🎉 Generated {len(validated_ideas)} valid blog ideas")
            
            # Convert back to dict for state storage
            ideas_as_dicts = [idea.dict() for idea in validated_ideas]
            
            return {
                **state,
                "raw_blog_ideas": ideas_as_dicts,
                "status": "ideas_generated"
            }
        else:
            return {**state, "error": "No valid ideas after validation", "status": "error"}
            
    except Exception as e:
        print(f"❌ Creative agent error: {e}")
        import traceback
        traceback.print_exc()
        return {**state, "error": str(e), "status": "error"}

print("✅ Creative agent node RELOADED")

✅ Creative agent node RELOADED


In [18]:
# Cell 19: Analyst Agent Node - FIXED for Pydantic Objects
def analyst_agent_node(state: AudioPipelineState) -> AudioPipelineState:
    """
    LangGraph node that scores blog ideas using company strategy context
    Input: state["raw_blog_ideas"] 
    Output: state["scored_blog_ideas"]
    """
    
    try:
        print("🔍 Starting analyst agent - scoring blog ideas...")
        
        # Check current status
        current_status = state.get('status', '')
        print(f"📊 Input status: {current_status}")
        
        # Check if we have raw blog ideas to score
        raw_ideas = state.get('raw_blog_ideas')
        if not raw_ideas:
            return {
                **state,
                "error": "No raw blog ideas available for scoring",
                "status": "error"
            }
        
        print(f"📊 Found {len(raw_ideas)} blog ideas to score")
        
        # Load strategy context for scoring
        print("📚 Loading strategy context...")
        strategy_context = prepare_strategy_context_for_scoring()
        
        # Get conversation context for better scoring
        conversation_context = state.get('transcript_text', '')
        
        # Score each blog idea
        scored_ideas = []
        for i, idea in enumerate(raw_ideas, 1):
            # FIXED: Handle both Pydantic objects and dicts properly
            if hasattr(idea, 'title'):
                # It's a Pydantic object - convert to dict first
                idea_dict = idea.dict() if hasattr(idea, 'dict') else idea.__dict__
                title_preview = idea.title[:50]
            else:
                # It's already a dict
                idea_dict = idea
                title_preview = idea.get('title', 'No title')[:50]
            
            print(f"🔍 Scoring idea {i}/{len(raw_ideas)}: {title_preview}...")
            
            # Score the idea (now always working with dict)
            scores = score_blog_idea_with_llm(idea_dict, strategy_context, conversation_context)
            
            # Combine original idea with scores
            scored_idea = {
                **idea_dict,  # Original idea data (now definitely a dict)
                **scores      # Scoring data
            }
            
            scored_ideas.append(scored_idea)
            
            print(f"   ✅ Scored: {scores.get('total_score', 0)}/70 points")
        
        # Sort by total score (highest first)
        scored_ideas.sort(key=lambda x: x.get('total_score', 0), reverse=True)
        
        print(f"\n🎉 Analyst agent completed scoring!")
        print(f"📊 Scored {len(scored_ideas)} ideas")
        
        if scored_ideas:
            print(f"🏆 Top idea: '{scored_ideas[0].get('title', 'Unknown')[:50]}...' ({scored_ideas[0].get('total_score', 0)}/70)")
            print(f"📉 Lowest idea: '{scored_ideas[-1].get('title', 'Unknown')[:50]}...' ({scored_ideas[-1].get('total_score', 0)}/70)")
        
        return {
            **state,
            "scored_blog_ideas": scored_ideas,
            "status": "ideas_scored"
        }
        
    except Exception as e:
        print(f"❌ Error in analyst agent node: {e}")
        import traceback
        traceback.print_exc()
        
        return {
            **state,
            "error": f"Analyst agent error: {str(e)}",
            "status": "error"
        }

print("✅ Analyst agent node FIXED for Pydantic objects")

✅ Analyst agent node FIXED for Pydantic objects


In [19]:
# Cell 4: Test AssemblyAI Connection
# Configure AssemblyAI
aai.settings.api_key = os.getenv('ASSEMBLYAI_API_KEY')

# Test with a simple transcription (we'll use a file from temp folder)
def test_assemblyai_connection():
    """Test if AssemblyAI is working"""
    try:
        # Just test the API key is valid
        transcriber = aai.Transcriber()
        print("✅ AssemblyAI connection successful")
        return True
    except Exception as e:
        print(f"❌ AssemblyAI connection failed: {e}")
        return False

test_assemblyai_connection()

✅ AssemblyAI connection successful


True

In [20]:
# Cell 21: Updated Pipeline Builder - Now with 5 Nodes
def build_pipeline():
    """Build the complete LangGraph workflow: Audio → Transcribe → Save → Insights → Ideas → Scoring"""
    workflow = StateGraph(AudioPipelineState)
    
    # Add all 5 nodes
    workflow.add_node("transcribe", transcription_node)
    workflow.add_node("save_to_db", database_saver_node)  
    workflow.add_node("extract_insights", pain_extractor_node)
    workflow.add_node("creative_agent", creative_agent_node)
    workflow.add_node("analyst_agent", analyst_agent_node)  # NEW: Node 5
    
    # Chain them together
    workflow.add_edge("transcribe", "save_to_db")
    workflow.add_edge("save_to_db", "extract_insights")
    workflow.add_edge("extract_insights", "creative_agent")
    workflow.add_edge("creative_agent", "analyst_agent")     # NEW: Connect to analyst
    
    workflow.set_entry_point("transcribe")
    workflow.set_finish_point("analyst_agent")              # NEW: End with analyst
    
    return workflow.compile()

# Rebuild the pipeline with 5 nodes
pipeline = build_pipeline()
print("✅ LangGraph pipeline compiled (5 nodes: transcribe → save_to_db → extract_insights → creative_agent → analyst_agent)")

✅ LangGraph pipeline compiled (5 nodes: transcribe → save_to_db → extract_insights → creative_agent → analyst_agent)


In [21]:
# Cell 7: Setup Anthropic LLM for Insights Extraction (FIXED)


# Initialize Anthropic with correct model name
anthropic_key = os.getenv('ANTHROPIC_API_KEY')
if not anthropic_key:
    print("⚠️  ANTHROPIC_API_KEY not found in .env file")
    print("Please add: ANTHROPIC_API_KEY=your_key_here")
else:
    llm = ChatAnthropic(
        model="claude-haiku-4-5",  # ← Updated model name
        api_key=anthropic_key,
        temperature=0.1
    )
    print("✅ Anthropic LLM initialized with Claude Haiku 4.5")

✅ Anthropic LLM initialized with Claude Haiku 4.5


In [23]:
## 3. PainExtractor Node Implementation

# System prompt
PAIN_EXTRACTOR_SYSTEM_PROMPT = """
You are a UX researcher and business analyst for BigKids Automation. Your job is listening to transcripts from interviews with users and potential clients. 

You pay special attention to problems that users have regarding how their company is automating, using web apps and AI to save time and move towards a more ethical and sovereign tech infrastructure.

You will be given the transcript of an interview with a user or potential client.

Your task is to extract structured information about:
- Who is speaking and their role
- What this person cares about (values, priorities)
- Their main primary and secondary challenges
- How they are solving problems today
- Are there AI agents that can assist them?
- Their underlying psychological needs (using frameworks like NVC - Non-Violent Communication)

Focus on automation, web apps, AI, time-saving, ethical tech, and sovereign infrastructure themes.

Be thorough but concise. 

IMPORTANT: Only extract information that is explicitly mentioned in the transcript. 
If information is not clearly stated, leave the field empty/null rather than guessing or inferring.
Do not hallucinate or make assumptions about missing information.
"""

In [24]:
# Cell 16: Enhanced Strategy Context for Scoring (Updated for 3 Documents)
def prepare_strategy_context_for_scoring():
    """Prepare strategy context for scoring using all three strategy documents"""
    
    # Load all three strategy documents
    strategy_context = load_company_strategy_context()
    
    # Add scoring guidelines
    strategy_context["scoring_guidelines"] = """
    SCORING CRITERIA (1-10 scale):
    
    1. usefulness_potential: How useful will this post be to readers with problems?
    2. fitwith_seo_strategy: How well does this align with our SEO strategy and keywords?
    3. fitwith_content_strategy: How well does this fit our content strategy and voice?
    4. inspiration_potential: How likely is this to inspire readers to take action?
    5. collaboration_potential: How likely is this to encourage prospects to contact us?
    6. innovation: How unique/differentiated is this topic (10 = very unique)?
    7. difficulty: How complex is this to write (1 = very complex, 10 = easy)?
    """
    
    # Create summaries for LLM prompt efficiency (all three documents)
    if strategy_context.get('company_strategy'):
        strategy_context["company_strategy_summary"] = strategy_context['company_strategy'][:800] + "..."
    
    if strategy_context.get('seo_strategy'):
        strategy_context["seo_strategy_summary"] = strategy_context['seo_strategy'][:600] + "..."
    
    if strategy_context.get('content_strategy'):  # NEW
        strategy_context["content_strategy_summary"] = strategy_context['content_strategy'][:600] + "..."
    
    print(f"✅ Enhanced strategy context for scoring with 3 documents")
    print(f"   Company strategy: {len(strategy_context.get('company_strategy', ''))} chars")
    print(f"   SEO strategy: {len(strategy_context.get('seo_strategy', ''))} chars")
    print(f"   Content strategy: {len(strategy_context.get('content_strategy', ''))} chars")
    
    return strategy_context

# Test the enhanced context
enhanced_context = prepare_strategy_context_for_scoring()

✅ Loaded company strategy (6555 chars)
✅ Loaded SEO strategy (1120 chars)
✅ Loaded content strategy (4469 chars)
✅ Enhanced strategy context for scoring with 3 documents
   Company strategy: 6555 chars
   SEO strategy: 1120 chars
   Content strategy: 4469 chars


In [25]:
# Cell: Fixed Test Function (No Duplicate Loading)
def test_three_document_loading():
    """Test loading all three strategy documents (optimized)"""
    
    print("🧪 Testing three-document strategy loading...")
    
    # Load documents once and enhance
    enhanced = prepare_strategy_context_for_scoring()  # This calls load_company_strategy_context() internally
    
    print(f"\n📊 DOCUMENT SUMMARY:")
    for doc_type in ['company_strategy', 'seo_strategy', 'content_strategy']:
        if doc_type in enhanced:
            length = len(enhanced[doc_type]) if enhanced[doc_type] else 0
            status = "✅ Loaded" if length > 100 else "⚠️ Missing/Short"
            print(f"   {doc_type}: {status} ({length} chars)")
    
    return enhanced

# Test with no duplicates
test_context = test_three_document_loading()

🧪 Testing three-document strategy loading...
✅ Loaded company strategy (6555 chars)
✅ Loaded SEO strategy (1120 chars)
✅ Loaded content strategy (4469 chars)
✅ Enhanced strategy context for scoring with 3 documents
   Company strategy: 6555 chars
   SEO strategy: 1120 chars
   Content strategy: 4469 chars

📊 DOCUMENT SUMMARY:
   company_strategy: ✅ Loaded (6555 chars)
   seo_strategy: ✅ Loaded (1120 chars)
   content_strategy: ✅ Loaded (4469 chars)


In [26]:
# Cell: Enhanced JSON Cleaning for Analyst Agent
def score_blog_idea_with_llm(idea: dict, strategy_context: dict, conversation_context: str = "") -> dict:
    """Score a single blog idea using LLM with enhanced JSON cleaning"""
    
    # ... existing prompt code stays the same ...
    
    try:
        response = llm.invoke(scoring_prompt)
        
        # ENHANCED: More robust JSON cleaning
        content = response.content.strip()
        
        # Remove markdown code blocks
        if content.startswith('```json'):
            content = content.replace('```json', '').replace('```', '').strip()
        
        # ENHANCED: Extract just the JSON part if there's extra content
        try:
            # Try to find JSON boundaries
            json_start = content.find('{')
            json_end = content.rfind('}') + 1
            
            if json_start != -1 and json_end > json_start:
                json_content = content[json_start:json_end]
                scores = json.loads(json_content)
            else:
                scores = json.loads(content)  # Fallback to original
                
        except json.JSONDecodeError:
            # If still failing, try to extract just the first complete JSON object
            import re
            json_match = re.search(r'\{.*?\}', content, re.DOTALL)
            if json_match:
                scores = json.loads(json_match.group())
            else:
                raise  # Re-raise the original error
        
        # ... rest of validation code stays the same ...
        
    except Exception as e:
        print(f"❌ Error scoring idea: {e}")
        return {
            "usefulness_potential": 5, "fitwith_seo_strategy": 5, "fitwith_content_strategy": 5,
            "inspiration_potential": 5, "collaboration_potential": 5, "innovation": 5,
            "difficulty": 5, "total_score": 35, "reasoning": f"Default scores due to error: {str(e)}"
        }

print("✅ Enhanced JSON cleaning for more robust scoring")

✅ Enhanced JSON cleaning for more robust scoring


In [30]:
# ============================================================
# CELL 19: EXECUTE COMPLETE 5-NODE PIPELINE TEST
# ============================================================

def test_complete_5_node_pipeline():
    """Test the complete 5-node pipeline: Audio → Transcribe → Save → Insights → Ideas → Scoring"""
    
    print("🧪 EXECUTING COMPLETE 5-NODE PIPELINE TEST...")
    print("Audio → Transcribe → Save → Insights → Ideas → Scoring")
    print("-" * 60)
    
    # Find audio files
    audio_files = find_audio_files_in_temp()
    
    if not audio_files:
        print("❌ No audio files found in data/temp/")
        print("💡 Add .wav files to data/temp/ and run this cell again")
        return None
    
    print(f"📁 Found {len(audio_files)} audio file(s)")
    print(f"🎯 Testing with: {audio_files[0].name}")
    print(f"📊 File size: {audio_files[0].stat().st_size / 1024:.1f} KB")
    
    # Create initial state for 5-node pipeline
    initial_state = {
        "file_path": str(audio_files[0]),
        "filename": audio_files[0].name,
        "transcript_text": None,
        "conversation_id": None,
        "extracted_insights": None,
        "raw_blog_ideas": None,
        "scored_blog_ideas": None,
        "error": None,
        "status": "processing"
    }
    
    try:
        print("\n🎬 STARTING COMPLETE PIPELINE EXECUTION...")
        print("=" * 60)
        
        # Run the complete 5-node pipeline
        final_state = pipeline.invoke(initial_state)
        
        print(f"\n📊 COMPLETE PIPELINE RESULTS:")
        print("=" * 60)
        print(f"🎯 Final Status: {final_state.get('status')}")
        print(f"📝 Conversation ID: {final_state.get('conversation_id')}")
        
        # Check all pipeline stages
        print(f"\n📋 STAGE RESULTS:")
        stages = [
            ("🎙️  Transcription", final_state.get('transcript_text')),
            ("💾 Database Save", final_state.get('conversation_id')),
            ("🧠 Insights Extraction", final_state.get('extracted_insights')),
            ("🎨 Blog Ideas Generation", final_state.get('raw_blog_ideas')),
            ("🔍 Blog Ideas Scoring", final_state.get('scored_blog_ideas'))
        ]
        
        all_passed = True
        for stage_name, stage_data in stages:
            status = "✅" if stage_data else "❌"
            print(f"   {stage_name}: {status}")
            if not stage_data:
                all_passed = False
        
        # Show detailed results if all stages passed
        if all_passed and final_state.get('scored_blog_ideas'):
            scored_ideas = final_state['scored_blog_ideas']
            
            print(f"\n🎉 COMPLETE SUCCESS! Pipeline generated and scored {len(scored_ideas)} blog ideas")
            print("=" * 80)
            
            # Show conversation info
            print(f"\n📝 Conversation Details:")
            print(f"   ID: {final_state.get('conversation_id')}")
            print(f"   Transcript Length: {len(final_state.get('transcript_text', ''))} characters")
            
            # Show insights summary
            if final_state.get('extracted_insights'):
                insights = final_state['extracted_insights']
                print(f"\n🧠 Extracted Insights:")
                print(f"   👥 Speakers: {len(insights.speakers) if hasattr(insights, 'speakers') else 'N/A'}")
                print(f"   🔥 Primary Challenges: {len(insights.primary_challenges) if hasattr(insights, 'primary_challenges') else 'N/A'}")
                print(f"   🧘 Psychological Needs: {len(insights.psychological_needs) if hasattr(insights, 'psychological_needs') else 'N/A'}")
                print(f"   💎 Core Values: {len(insights.core_values) if hasattr(insights, 'core_values') else 'N/A'}")
            
            # Show top scored ideas
            print(f"\n🎨 Top Scored Blog Ideas:")
            print("-" * 80)
            
            for i, idea in enumerate(scored_ideas[:3], 1):  # Show top 3
                print(f"\n🏆 RANKED IDEA #{i} (Total Score: {idea.get('total_score', 0)}/70):")
                print(f"   📝 Title: {idea.get('title', 'No title')[:75]}...")
                print(f"   📊 Breakdown:")
                print(f"      • Usefulness: {idea.get('usefulness_potential', 0)}/10")
                print(f"      • SEO Fit: {idea.get('fitwith_seo_strategy', 0)}/10")
                print(f"      • Content Fit: {idea.get('fitwith_content_strategy', 0)}/10")
                print(f"      • Inspiration: {idea.get('inspiration_potential', 0)}/10")
                print(f"      • Collaboration: {idea.get('collaboration_potential', 0)}/10")
                print(f"      • Innovation: {idea.get('innovation', 0)}/10")
                print(f"      • Easy to Write: {idea.get('difficulty', 0)}/10")
            
            print("=" * 80)
            print("🎉 COMPLETE 5-NODE PIPELINE: SUCCESS!")
            print("✅ All stages completed successfully")
            print(f"💾 Data saved to database (conversation_id: {final_state.get('conversation_id')})")
            
        else:
            # Something failed
            print("\n❌ PIPELINE INCOMPLETE")
            print("=" * 50)
            if final_state.get('error'):
                print(f"❌ Error: {final_state.get('error')}")
            else:
                print("❌ Pipeline stopped but no error message provided")
            
            # Show what we did get
            print(f"\n🔍 DEBUG INFO:")
            print(f"   Transcript exists: {bool(final_state.get('transcript_text'))}")
            if final_state.get('transcript_text'):
                print(f"   Transcript preview: {final_state.get('transcript_text')[:100]}...")
            print(f"   Insights exist: {bool(final_state.get('extracted_insights'))}")
            print(f"   Raw ideas exist: {bool(final_state.get('raw_blog_ideas'))}")
            print(f"   Scored ideas exist: {bool(final_state.get('scored_blog_ideas'))}")
        
        return final_state
        
    except Exception as e:
        print(f"\n❌ COMPLETE PIPELINE FAILED WITH EXCEPTION:")
        print(f"   {str(e)}")
        import traceback
        traceback.print_exc()
        return None

# ============================================================
# RUN THE TEST
# ============================================================

print("🚀 Ready to test complete 5-node pipeline")
print("💡 Run the cell below to execute the test\n")

# Uncomment the line below to run automatically, or run it manually
test_result = test_complete_5_node_pipeline()

🚀 Ready to test complete 5-node pipeline
💡 Run the cell below to execute the test

🧪 EXECUTING COMPLETE 5-NODE PIPELINE TEST...
Audio → Transcribe → Save → Insights → Ideas → Scoring
------------------------------------------------------------
📁 Found 1 audio file(s)
🎯 Testing with: blog_record_(purevitalize).wav
📊 File size: 25016.7 KB

🎬 STARTING COMPLETE PIPELINE EXECUTION...
🎙️ Transcribing: blog_record_(purevitalize).wav
💾 Saving to database: blog_record_(purevitalize).wav
🧠 Starting pain extraction...
📝 Raw response length: 3629 chars
📝 Response starts with: ```json
{
    "speakers": [
        {
            ...
🔧 Removing JSON markdown blocks...
🔧 Cleaned content starts with: {
    "speakers": [
        {
            "name": ...
✅ Successfully extracted insights with correct speaker roles!
✅ Extracted insights: 3 primary challenges, 1 speakers
🎨 Starting creative blog idea generation...
📊 Working with insights: 3 challenges
✅ Loaded company strategy (6555 chars)
✅ Loaded SEO stra

/tmp/ipykernel_9844/4162762669.py:36: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  ideas_as_dicts = [idea.dict() for idea in validated_ideas]


In [28]:
# Cell: Rebuild Pipeline with Updated Node
pipeline = build_pipeline()
print("✅ Pipeline rebuilt with updated creative agent")

✅ Pipeline rebuilt with updated creative agent


In [ ]:
# Cell: Test Fixed Pain Extraction
def test_pain_extraction_fix():
    """Test the fixed pain extraction with your transcript"""
    
    # Get the conversation that just failed
    conversations = db.get_all_conversations()
    latest_conversation = conversations[0] if conversations else None
    
    if latest_conversation and latest_conversation.raw_text:
        print("🧪 Testing fixed pain extraction...")
        print(f"📝 Using conversation: {latest_conversation.title}")
        
        try:
            # Test the fixed extraction
            insights = extract_insights_from_transcript(latest_conversation.raw_text)
            print(f"✅ Success! Extracted {len(insights.primary_challenges)} challenges")
            print(f"👥 Found {len(insights.speakers)} speakers")
            return True
        except Exception as e:
            print(f"❌ Still failing: {e}")
            return False
    else:
        print("❌ No conversation found to test with")
        return False

# Test the fix
test_pain_extraction_fix()